In [1]:
!gcc zhunt3-alan.c -lm -o zhunt3
!chmod a+x zhunt3

zhunt3-alan.c: In function ‘user_regret’:
zhunt3-alan.c:303:7: warning: implicit declaration of function ‘gets’; did you mean ‘fgets’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  303 |       gets( tempstr );
      |       ^~~~
      |       fgets
/usr/bin/ld: /tmp/ccgHgRNb.o: in function `user_regret':
zhunt3-alan.c:(.text+0xbdb): warning: the `gets' function is dangerous and should not be used.


In [2]:
!./zhunt3 --help

usage: zhunt windowsize minsize maxsize datafile


In [1]:
file = "../ncbi_dataset/data/GCF_000787575.1/GCF_000787575.1_Asub_2.0_genomic.fna"
from Bio import SeqIO

# divide fna file into separate file scaffold wise
# create new directory for them
import os

os.makedirs("scaffolds", exist_ok=True)

for record in SeqIO.parse(file, "fasta"):
    with open(f"scaffolds/{record.id}.fna", "w") as f:
        SeqIO.write(record, f, "fasta")


In [2]:
# run zhunt3 on all files in scaffolds directory in parallel (8 threads)
import os
import subprocess
import concurrent.futures

def run_zhunt3(file):
    subprocess.run(["./zhunt3", "12", "8", "12", f"scaffolds/{file}"])

files = os.listdir("scaffolds")

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    # Use the executor to map the function to the files
    executor.map(run_zhunt3, files)

dinucleotides 12
min/max 8 12
min/max 8 12
operating on scaffolds/NW_012236428.1.fna
calculating zscore
opening scaffolds/NW_012236428.1.fna
inputting sequence
opening scaffolds/NW_012236428.1.fna.Z-SCORE

 run time=0 sec
use min/max 8 12
analyzing_zscore
opening scaffolds/NW_012236428.1.fna.Z-SCORE
opening scaffolds/NW_012236428.1.fna
inputting sequence
dinucleotides 12
min/max 8 12
min/max 8 12
operating on scaffolds/NW_012236447.1.fna
calculating zscore
opening scaffolds/NW_012236447.1.fna
inputting sequence
opening scaffolds/NW_012236447.1.fna.Z-SCORE

 run time=1 sec
use min/max 8 12
analyzing_zscore
opening scaffolds/NW_012236447.1.fna.Z-SCORE
opening scaffolds/NW_012236447.1.fna
inputting sequence
dinucleotides 12
min/max 8 12
min/max 8 12
operating on scaffolds/NW_012236457.1.fna
calculating zscore
opening scaffolds/NW_012236457.1.fna
inputting sequence
opening scaffolds/NW_012236457.1.fna.Z-SCORE

 run time=1 sec
use min/max 8 12
analyzing_zscore
opening scaffolds/NW_012236457

In [6]:
import pandas as pd
def combine_overlapping_regions(zhunt_filtered) -> pd.DataFrame:
    zhunt_filtered = zhunt_filtered.sort_values("Start")
    zhunt_filtered = zhunt_filtered.reset_index(drop=True)
    i = 0
    while i < len(zhunt_filtered) - 1:
        if zhunt_filtered.loc[i, "End"] >= zhunt_filtered.loc[i+1, "Start"]:
            zhunt_filtered.loc[i, "End"] = max(zhunt_filtered.loc[i, "End"], zhunt_filtered.loc[i+1, "End"])
            zhunt_filtered = zhunt_filtered.drop(i+1)
            zhunt_filtered = zhunt_filtered.reset_index(drop=True)
        else:
            i += 1
    return zhunt_filtered

In [7]:
# combine all the output files into one bed file with columns as scaffold, start, end, score
# output files have the same name as input files with .Z-SCORE extension
# 

import pandas as pd


# all_files = os.listdir("scaffolds")
# output_files = [file for file in all_files if file.endswith(".Z-SCORE")]

final = pd.DataFrame(columns=["Scaffold","Start","End","Score"])

for record in SeqIO.parse(file, "fasta"):
    scaffold = record.id
    output_file_name = f"scaffolds/{scaffold}.fna.Z-SCORE"
    zhunt = pd.read_csv(output_file_name, skiprows=1, names=["Start","End","1","2","3","Score","Seq","4"], delim_whitespace=True)
    zhunt["Scaffold"] = scaffold
    zhunt = zhunt[["Scaffold","Start","End","Score"]]
    # filter by score
    zhunt = zhunt[zhunt["Score"] > 1000]
    # combine regions if they overlap
    zhunt = combine_overlapping_regions(zhunt)
    
    final = pd.concat([final, zhunt])

/tmp/ipykernel_109819/2412485415.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  zhunt = pd.read_csv(output_file_name, skiprows=1, names=["Start","End","1","2","3","Score","Seq","4"], delim_whitespace=True)
/tmp/ipykernel_109819/2412485415.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  zhunt = pd.read_csv(output_file_name, skiprows=1, names=["Start","End","1","2","3","Score","Seq","4"], delim_whitespace=True)
/tmp/ipykernel_109819/2412485415.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat([final, zhunt])
/tmp/ipyke

In [8]:
final.head(10)

,Scaffold,Start,End,Score
0,NW_012236428.1,731,761,1237.247
1,NW_012236428.1,1359,1382,1096.390
0,NW_012236431.1,1627,1664,1440.850
0,NW_012236432.1,44,72,2539.351
0,NW_012236435.1,2125,2149,1030.985
0,NW_012236444.1,2173,2203,1237.247
0,NW_012236445.1,752,772,1161.851
0,NW_012236456.1,3375,3403,1113.221
0,NW_012236465.1,3401,3421,1091.395
0,NW_012236466.1,1190,1210,1161.851


In [11]:
len(final)

4328

In [9]:
# save the final dataframe to a bed file
final.to_csv("zhunt.bed", sep="\t", index=False, header=False)

In [ ]:
# !./zhunt3 12 8 12 "../ncbi_dataset/data/GCF_000787575.1/GCF_000787575.1_Asub_2.0_genomic.fna"

dinucleotides 12
min/max 8 12
min/max 8 12
operating on ../ncbi_dataset/data/GCF_000787575.1/GCF_000787575.1_Asub_2.0_genomic.fna
calculating zscore
opening ../ncbi_dataset/data/GCF_000787575.1/GCF_000787575.1_Asub_2.0_genomic.fna
inputting sequence
opening ../ncbi_dataset/data/GCF_000787575.1/GCF_000787575.1_Asub_2.0_genomic.fna.Z-SCORE
